In [15]:
import os
from tqdm import tqdm
from datasets import load_dataset
import sys
sys.path.append('../evaluation')
from evaluate import RetrievalSystem, main as evaluate_main
import yaml
import hyde

In [4]:
with open("/users/christineye/retrieval/config.yaml", 'r') as stream:
    api_key = yaml.safe_load(stream)['openai_api_key']

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
test_hyde = hyde.HydeRetrievalSystem(config_path = "/users/christineye/retrieval/config.yaml", 
                                     db_path = "", generate_n = 3, embed_query = True, max_doclen = 100)

In [31]:
test_hyde.retrieve("What are the types of Type 1a supernovae?")

Abstract:
The progenitors of Type Ia supernovae (SNe Ia) remain a subject of debate. We investigate the various proposed models for SNe Ia, including the single-degenerate and double-degenerate scenarios, as well as the possibility of sub-Chandrasekhar mass explosions.

Conclusion:
Our analysis suggests that multiple pathways may lead to SNe Ia, including the accretion of material onto a white
[0.035977237, 0.03236715, 0.0384499, -0.046758045, 0.014452712, -0.015145057, 0.037881188, -0.0025004798, -0.007417987, 0.0070718145, -0.0070161796, 0.0073932605, -0.062360544, 0.008901585, 0.0010786989, -0.020510735, -0.040329125, 0.014106539, 0.010694264, 0.021920152, -0.024479358, -0.014588708, 0.04767293, -0.009457934, 0.012845481, -0.005439857, 0.020683821, 0.029845035, 0.06171765, -0.006768913, 0.02042419, -0.003594633, -0.050120864, -0.00391917, -0.063003436, 0.06077804, 0.028213078, 0.0064536487, -0.01683883, 0.00560058, 0.01206041, -0.022785584, -0.009414662, 0.07200393, -0.011337157, 0.

IndexError: list index out of range